<h1 align="center">CS340 - Assignment 2</h1>
<h3 align="center">Due Date: 18 April 2017</h3>
<br>
<p style="text-indent: 40px">In this project we are going to work on movielens 20m dataset with <b>PySpark Dataframes</b>. 
The dataset is included in the assignment folder. You should unzip it and upload each file to the ibm's datascience tool.
</p>

In order to understand the data in each csv file, you should read README.txt file inside the folder.

## Reliable Users

<p style="text-indent: 40px">For the first part of this assignment you should find out the top 1000 reliable users. In order to do that you would need tags' relevance score from genome-scores.csv file. Assuming the relevance scores of movies' tags in genome-scores.csv are correct, we can find out if a user's tag to a particular movie is relevant or not. If we get the total of those relevance scores for each user then we can rank the them according to this criteria.</p>

<p style="text-indent: 40px">One caveat is that relevance scores are between 0 and 1. We want to scale it between -1 and 1 and also we want to punish more if the issued tag is not relevant. First multiply relevance with 20 and then substract 12 so relevance score range will be between -12 and 8 and then apply hyperbolic tangent. That way relevance's range will be between -1 and 1.</p>

<p style="text-indent: 40px"> Note: You should use PySpark's <b>tanh</b> function. Also use caching appropriately in order to be able to get the results in a reasonable time.</p>
<p style="text-indent: 40px"> Caution: Do not use RDDs. Every operation should be done on the executors not in the driver machine.</p>
<p style="text-indent: 40px">Important: Do not discuss the solution with your friends. <b>Plagiarism</b> will not be tolerated and issue will be referred to the <b>disciplinary committee</b>.</p>

<div>
    <img src="http://image.prntscr.com/image/1f91352835964c31b04be03da7d53581.jpg" width=500>

    <center><strong>Figure 1: Tanh function.</strong></center>
</div>

In [1]:

from pyspark.sql import SQLContext
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_3a500b3a65a940508dadf71c13437e1d(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '266ba3381db2401faf8aaa2af05a9146')
    hconf.set(prefix + '.username', 'e6181ca8d25046b5a8f4ee43c04b79b8')
    hconf.set(prefix + '.password', 'qK-mWT(9O,q89]cY')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_3a500b3a65a940508dadf71c13437e1d(name)


In [2]:
tags_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/tags.csv")

In [3]:
genome_tags_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/genome-tags.csv")

In [4]:
genome_scores_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/genome-scores.csv")

In [5]:
ratings_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/ratings.csv")

In [6]:
movies_df = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://CS340." + name + "/movies.csv")

In [7]:
userid_movieid_tags = tags_df.select("userId","movieId","tag")
# userid_movieid_tags.show()

In [8]:
userid_movieid_tagId = userid_movieid_tags.join(genome_tags_df, genome_tags_df.tag == userid_movieid_tags.tag).select("userId","movieId","tagId")
userid_movieid_tagId.show()

+------+-------+-----+
|userId|movieId|tagId|
+------+-------+-----+
| 70279|  68358|   29|
| 70463|   2959|   29|
| 70515|    480|   29|
| 70592|  98809|   29|
| 70592| 106489|   29|
| 70833|    480|   29|
| 70911|   7438|   29|
| 71328|  34405|   29|
| 71328|  77866|   29|
| 71328|  78105|   29|
| 71328|  79185|   29|
| 71328| 106489|   29|
| 71432|   1208|   29|
| 71432|  48394|   29|
| 71432|  77866|   29|
| 71544|   2470|   29|
| 71887|  76175|   29|
| 71929|   1208|   29|
| 71939|  77866|   29|
| 71939|  85414|   29|
+------+-------+-----+
only showing top 20 rows



In [9]:
userId_reliability = userid_movieid_tagId.join(genome_scores_df, (userid_movieid_tagId.movieId == genome_scores_df.movieId)\
                                                           & (userid_movieid_tagId.tagId == genome_scores_df.tagId))
userId_reliability = userId_reliability.select("userId", tanh(userId_reliability["relevance"]*20 - 12))
userId_reliability = userId_reliability.groupBy("userId").sum(userId_reliability.columns[-1])
userId_reliability = userId_reliability.select(col(userId_reliability.columns[0]).alias("userId"), col(userId_reliability.columns[-1]).alias("reliability"))
userId_reliability.show()

+------+--------------------+
|userId|         reliability|
+------+--------------------+
|  6431|   665.8821765083229|
| 29831|   81.99768511072358|
|106831|   2.340801667606085|
|   631|   33.78900572273493|
| 86231|  1.9999974957380933|
|121631|  29.840888210409545|
| 54431|  11.783470565361455|
|117831|  5.2067628965686374|
| 46431|   17.06159530430593|
| 26831|    46.9825301538157|
|111231|   81.11620036999845|
| 16231|  37.999985848894546|
| 47031| -0.9999006540637964|
|  3031|   69.34044259878408|
|124431|  1.0957575120283989|
| 46231|   4.833363991053533|
| 83631|  14.862813995762817|
| 99431|  0.9999995557387478|
|137031|-1.26206749930912...|
|134031|  3.3841913131454553|
+------+--------------------+
only showing top 20 rows



In [10]:
top_thousand_users = userId_reliability.orderBy(col("reliability").desc()).limit(1000)
# assert top_thousand_users.count() == 1000
top_thousand_users.show()

+------+------------------+
|userId|       reliability|
+------+------------------+
| 88738| 5886.228605226696|
| 58612|1700.3142105607255|
| 52814| 1648.838228869247|
| 11081|1582.5985838545291|
| 10616|1447.9943988488747|
|  9815|1413.2447981826167|
| 68558|1261.1889719076667|
|  1741|1119.2541784879347|
| 25737|1068.4360438182064|
|  4450|1000.3835964912713|
| 27898| 969.4690500707222|
|122523| 961.2716671956216|
|119367| 959.6825546999676|
| 77463| 927.4315434523476|
|124998| 854.3481069572276|
| 77297|  845.153160315068|
| 28906| 840.9469449647927|
|120937| 797.6761602947414|
|  6431| 665.8821765083231|
|111982| 662.9396577252996|
+------+------------------+
only showing top 20 rows



## Best Movies

<br>
<p style="text-indent: 40px">For the second part of this assignment, we are going to get best 20 movies from those 1000 people's scores. For each movie in the ratings, you should multiply rating with the users score that we previously calculated and get the total of this score for each movie.

In the end we should be able to see the titles of the first 20 movies ranked by this criteria.
</p>




In [11]:
userId_movieId_rating_reliability = ratings_df.select(ratings_df.columns[:-1]).join(top_thousand_users, top_thousand_users.userId == ratings_df.userId)\
                                    .select(ratings_df.userId, "movieId","rating","reliability")
userId_movieId_rating_reliability.show()

+------+-------+------+-----------------+
|userId|movieId|rating|      reliability|
+------+-------+------+-----------------+
|    65|     24|   4.0|25.71999218707428|
|    65|    318|   5.0|25.71999218707428|
|    65|    356|   5.0|25.71999218707428|
|    65|    364|   4.5|25.71999218707428|
|    65|    443|   4.0|25.71999218707428|
|    65|    480|   3.0|25.71999218707428|
|    65|    541|   4.0|25.71999218707428|
|    65|    588|   5.0|25.71999218707428|
|    65|   1148|   3.5|25.71999218707428|
|    65|   1197|   2.0|25.71999218707428|
|    65|   1235|   5.0|25.71999218707428|
|    65|   1250|   2.5|25.71999218707428|
|    65|   1271|   4.5|25.71999218707428|
|    65|   1348|   3.5|25.71999218707428|
|    65|   1653|   4.0|25.71999218707428|
|    65|   1688|   2.0|25.71999218707428|
|    65|   1704|   3.5|25.71999218707428|
|    65|   1801|   2.5|25.71999218707428|
|    65|   1921|   3.5|25.71999218707428|
|    65|   2114|   4.0|25.71999218707428|
+------+-------+------+-----------

In [12]:
movie_scores = userId_movieId_rating_reliability.withColumn("score", userId_movieId_rating_reliability.rating * userId_movieId_rating_reliability.reliability)\
                .selectExpr("movieId as movieId1","score as score").groupBy("movieId1").sum("score")
top_20_movies = movie_scores.select("movieId1",col(movie_scores.columns[-1]).alias("score"))\
                .join(movies_df, movie_scores.movieId1 == movies_df.movieId).select("movieId","title","score")\
    .orderBy(col("score").desc()).limit(20)
top_20_movies.show()

+-------+--------------------+------------------+
|movieId|               title|             score|
+-------+--------------------+------------------+
|   2571|  Matrix, The (1999)| 471354.5594368037|
|   2959|   Fight Club (1999)| 426152.0577531985|
|    296| Pulp Fiction (1994)|415367.08530751534|
|    318|Shawshank Redempt...| 409525.1721352335|
|    356| Forrest Gump (1994)|395281.52323548956|
|   4993|Lord of the Rings...| 395052.4140476214|
|   4226|      Memento (2000)|378664.83073142625|
|  79132|    Inception (2010)| 375745.9660923454|
|   5952|Lord of the Rings...|372978.42857394955|
|   7153|Lord of the Rings...|364419.68578229175|
|  58559|Dark Knight, The ...|361753.23284971487|
|    593|Silence of the La...|360313.65050738555|
|    260|Star Wars: Episod...|359782.00395323953|
|   2858|American Beauty (...| 358955.4184849296|
|   1270|Back to the Futur...|352814.61000777053|
|   7361|Eternal Sunshine ...| 351084.1205756394|
|   1196|Star Wars: Episod...| 349536.5973937372|
